# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
from IPython.display import Markdown, display, update_display
from dotenv import load_dotenv
from openai import OpenAI, Stream

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [8]:
# set up environment
load_dotenv(override=True)

from typing import List

def stream_response(stream: Stream) -> None:
    """
    Processes a Stream object and updates a Markdown display dynamically with content
    received from the stream chunks.

    Args:
        stream (Stream): An object that produces chunks of responses with choices.
    """
    response_parts: List[str] = []

    # Initialize the display with Markdown formatting
    handle_display = display(Markdown, display_id=True)

    for chunk in stream:
        # Ensure we extract content safely
        if chunk.choices and hasattr(chunk.choices[0], 'delta') and hasattr(chunk.choices[0].delta, 'content'):
            content = chunk.choices[0].delta.content or ''
            response_parts.append(content)
        else:
            logging.warning("Chunk has no choices or invalid structure.")

    # Update the response display once at the end
    full_response = ''.join(response_parts)
    update_display(Markdown(full_response), display_id=handle_display.display_id)

In [7]:
# here is the question; type over this to ask something new
code = """
def stream_response(stream: Stream) -> None:
    "/"/"/
    Processes a Stream object and updates a Markdown display dynamically with content
    received from the stream chunks.

    Args:
        stream (Stream): An object that produces chunks of responses with choices.
    "/"/"/
    response_parts: List = []

    # Initialize the display with Markdown formatting
    handle_display = display(Markdown, display_id=True)

    for chunk in stream:
        # Ensure we extract content safely
        try:
            content = chunk.choices[0].delta.content or ''
            response_parts.append(content)
        except (IndexError, AttributeError) as error:
            print(f"Error processing chunk: {/error}/")

    # Update the response display once at the end
    full_response = ''.join(response_parts)
    update_display(Markdown(full_response), display_id=handle_display.display_id)
"""

question = """
Please explain what this code does and why:

"""+code

system_prompt="""You are a helpful assistant that explains code.
 You want to explain the code in a way that is best practice for a developer and easy to learn.
 You use best practices for code readability and maintainability and also judge the code on readability and maintainability and clean code/architecture and give better code suggestions.
 Use only most recent knowledge and information.
"""

# construct the messages
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question}
]


In [ ]:
# Get gpt-4o-mini to answer, with streaming
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

stream = openai.chat.completions.create(
    messages=messages,
    model=MODEL,
    stream=True,

)

stream_response(stream)

In [9]:
# Get Llama 3.2 to answer
load_dotenv(override=True)
api_key = os.getenv('OLLAMA_API_KEY')
base_url= os.getenv('OLLAMA_BASE_URL')
print("base_url:", repr(base_url))
print("model:", MODEL_LLAMA)
openai = OpenAI(base_url=base_url, api_key=api_key)

stream = openai.chat.completions.create(
    model=MODEL_LLAMA,
    messages=messages,
    stream=True
)

stream_response(stream)

base_url: 'http://localhost:11434/v1'
model: llama3.2


**Code Review and Explanation**

The provided code defines a function `stream_response` that processes a stream of chunks, extracts relevant content from each chunk, and updates a Markdown display dynamically.

### Purpose:

This function appears to be part of a larger application, likely related to web development or interactive tools, where the user interacts with a Stream-like object to receive responses. The goal is to capture these responses and update a Markdown-based display in real-time.

**Best Practices**

Here are some observations about the code's readability, maintainability, and overall quality:

*   **Docstrings**: The docstring provides sufficient context for users who might need to understand this function. However, it would be ideal to include more detailed information about the expected input formats (e.g., `stream` type), error handling specifics, or any specific requirements related to display updates.
*   **Variable naming**: Variable names like `response_parts`, `handle_display`, and `display_id` could be improved with more descriptive names. Consider using a consistent naming convention throughout the function, such as following PEP 8's guidance for variable names in Python.
*   **Error handling**: While the code catches and handles the two types of exceptions that might occur during chunk processing (`IndexError` and `AttributeError`), it simply prints an error message to the console. Consider logging or providing more informative error messages, potentially allowing for additional debugging information if needed.
*   **Code organization**: This function performs both data extraction and display updates. Break down this logic into separate functions to improve readability and maintainability.

### Suggested Improvements and Alternative Implementations

Here are some code improvements:

*   **Process chunk extraction more robustly**

    ```python
def process_chunk(chunk):
    """Extracts content from a stream chunk"""
    return chunk.choices[0].delta.content or ''
```

*   **Create separate functions for setup and display updates**

    ```python
def prepare_response_display():
    """Sets up the Markdown response display."""
    handle_display = display(Markdown, display_id=True)
    return handle_display
```

    ```python
def update_markdown_display(text, display_id):
    """Updates the Markdown display with given text."""
    # Update logic goes here
    pass
```

*   **Reconsider the approach for displaying updates**

    Instead of directly creating and updating a Markdown display in-place, consider using HTML elements to render dynamic content, and update those elements as needed.

Here's an updated, refactored version of your code with these suggestions integrated:

```python
# Setup function
def prepare_response_display():
    """Sets up the Markdown response display."""
    handle_display = display(Markdown, display_id=True)
    return handle_display

def stream_response(stream):
    """Processes a Stream object and updates a Markdown display dynamically."""
    
    response_parts: List[str] = []

    for chunk in stream:
        try:
            content = process_chunk(chunk)
            response_parts.append(content)
        except (IndexError, AttributeError) as error:
            print(f"Error processing chunk: {str(error)}")

    # Update the display
    full_response_text = ''.join(response_parts)
    update_markdown_display(full_response_text, handle_display.display_id)

def process_chunk(chunk):
    """Extracts content from a stream chunk"""
    return chunk.choices[0].delta.content or ''

def update_markdown_display(text: str, display_id):
    # Update logic goes here
    pass
```

Note that the `display` function and `update_display` functions are hypothetical since they were not included in your original code snippet. The refactored version assumes their existence, but you should implement these functions according to your application's specific requirements.